In [6]:
def get_df_from_json(data_url, record_path=['Data']):
    content = requests.get(data_url).json()
    df_all = pd.json_normalize(content,record_path=record_path)
    
    return df_all

In [7]:
# Scrape webpage
import io
import requests
import pandas as pd

# Start date and end date
start_date = '2022-10-14'
end_date = '2022-10-15'



In [9]:
# Get river data
river_data_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=35946&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
flow_df_all = get_df_from_json(river_data_url)
flow_df = flow_df_all[["TimeStamp", "Value"]]
flow_df = flow_df.rename(columns={'Value': 'FlowRate'})
flow_df['TimeStamp']= pd.to_datetime(flow_df['TimeStamp'])

print(flow_df.shape)
print(flow_df.head())

(289, 2)
                  TimeStamp   FlowRate
0 2022-10-15 00:00:00+00:00  40.598459
1 2022-10-14 23:55:00+00:00  40.598459
2 2022-10-14 23:50:00+00:00  40.429302
3 2022-10-14 23:45:00+00:00  40.361712
4 2022-10-14 23:40:00+00:00  40.159192


In [10]:
# Get Lake levels
lake_level_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=32419&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
lake_level_df_all = get_df_from_json(lake_level_url)
lake_level_df = lake_level_df_all[["TimeStamp", "Value"]]
lake_level_df = lake_level_df.rename(columns={'Value': 'LakeLevel'})
lake_level_df['TimeStamp']= pd.to_datetime(lake_level_df['TimeStamp'])
print(lake_level_df.shape)
print(lake_level_df.head())


(289, 2)
                  TimeStamp  LakeLevel
0 2022-10-15 00:00:00+00:00    279.221
1 2022-10-14 23:55:00+00:00    279.220
2 2022-10-14 23:50:00+00:00    279.221
3 2022-10-14 23:45:00+00:00    279.221
4 2022-10-14 23:40:00+00:00    279.223


In [11]:
# Get rainfall at Lake Rotoiti

rainfall_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=47928&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
rainfall_df_all = get_df_from_json(rainfall_url)
rainfall_df = rainfall_df_all[["TimeStamp", "Value"]]
rainfall_df = rainfall_df.rename(columns={'Value': 'Rainfall'})
rainfall_df['TimeStamp']= pd.to_datetime(rainfall_df['TimeStamp'])
print(rainfall_df.head())

                  TimeStamp  Rainfall
0 2022-10-15 00:00:00+00:00       0.0
1 2022-10-14 23:00:00+00:00       0.0
2 2022-10-14 22:00:00+00:00       0.0
3 2022-10-14 21:00:00+00:00       0.0
4 2022-10-14 20:00:00+00:00       0.0


In [21]:
# Merge datasets into one dataframe
df_temp = pd.merge(lake_level_df, flow_df, on='TimeStamp')
df_combined = pd.merge(df_temp, rainfall_df, on='TimeStamp', how='left')

#Set index
df_combined.index = df_all['TimeStamp']

#Remove rows for which we don't have rainfall data
df_river_data = df_combined.dropna()
print (df_river_data)

                                          TimeStamp  LakeLevel   FlowRate  \
TimeStamp                                                                   
2022-10-15 00:00:00+00:00 2022-10-15 00:00:00+00:00    279.221  40.598459   
2022-10-14 23:00:00+00:00 2022-10-14 23:00:00+00:00    279.223  40.294164   
2022-10-14 22:00:00+00:00 2022-10-14 22:00:00+00:00    279.225  40.632322   
2022-10-14 21:00:00+00:00 2022-10-14 21:00:00+00:00    279.225  40.429302   
2022-10-14 20:00:00+00:00 2022-10-14 20:00:00+00:00    279.223  40.463113   
2022-10-14 19:00:00+00:00 2022-10-14 19:00:00+00:00    279.229  40.666195   
2022-10-14 18:00:00+00:00 2022-10-14 18:00:00+00:00    279.227  40.666195   
2022-10-14 17:00:00+00:00 2022-10-14 17:00:00+00:00    279.228  40.598459   
2022-10-14 16:00:00+00:00 2022-10-14 16:00:00+00:00    279.231  40.632322   
2022-10-14 15:00:00+00:00 2022-10-14 15:00:00+00:00    279.229  40.700078   
2022-10-14 14:00:00+00:00 2022-10-14 14:00:00+00:00    279.233  40.632322   